In [1]:
import numpy as np

In [12]:
class BasicAnn:
    def __init__(self, layers, learning_rate):
        self.layers = layers
        self.learning_rate = learning_rate
        self.biases = [np.zeros((s,1)) for s in layers[1:]]
        self.weight_shapes = [(layers[i+1], layers[i]) for i in range(len(layers)-1)]
        self.weights = [np.random.standard_normal(shape) for shape in self.weight_shapes]
        
    def _sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def _sigmoid_deriv(self, x):
        return self._sigmoid(x) * (1 - self._sigmoid(x))
    
    def predict(self, input_vector):
        layer_1 = np.dot(input_vector, self.weights) + self.bias
        layer_2 = self._sigmoid(layer_1)
        return layer_2
    
myAnn = BasicAnn((2,3,5,2), 0.01)